# Bert text classification on SST2 using PyTorch





In [1]:
!pip install -r requirements_notebook.txt

In [2]:
import sys, os, shutil
import logging

sys.path.append("src")

logging.basicConfig(level="INFO", handlers=[logging.StreamHandler(sys.stdout)],
                        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

### Bucket and role set up

In [3]:
import sagemaker, boto3
from sagemaker import get_execution_role
sm_session = sagemaker.session.Session()


sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


2021-01-19 23:07:20,411 - sagemaker.analytics - WARNING - pandas failed to import. Analytics features will be impaired or broken.
2021-01-19 23:07:20,653 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2021-01-19 23:07:20,898 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2021-01-19 23:07:21,046 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [4]:
resume_from_checkpoint=False

In [5]:
data_bucket = sm_session.default_bucket()

data_bucket_prefix = "bert-sst2-classify"

s3_uri_data = "s3://{}/{}/data".format(data_bucket, data_bucket_prefix)
s3_uri_train = "{}/{}".format(s3_uri_data, "train.csv")
s3_uri_val = "{}/{}".format(s3_uri_data, "dev.csv")

s3_uri_test = "{}/{}".format(s3_uri_data, "test.csv")

s3_output_path = "s3://{}/{}/output".format(data_bucket, data_bucket_prefix)
s3_code_path = "s3://{}/{}/code".format(data_bucket, data_bucket_prefix)
s3_checkpoint = "s3://{}/{}/checkpoint".format(data_bucket, data_bucket_prefix)

In [6]:
import boto3

def delete_s3_objects(s3_uri):
    client = boto3.client('s3')
    s3_uri = s3_uri.replace("s3://","")
    bucket, prefix = s3_uri.split("/")[0], "/".join( s3_uri.split("/")[1:])
    
    response = client.list_objects(
    Bucket=bucket,
    Delimiter='|',
    Prefix=prefix,
    MaxKeys = 20
)
    s3 = boto3.resource('s3')
    for item in response.get("Contents", []):
        print("Deleting {}".format(item["Key"]))
        obj = s3.Object(bucket, item["Key"] )
        obj.delete()
    
    


In [7]:
if not resume_from_checkpoint:
    delete_s3_objects(s3_checkpoint)

Deleting bert-sst2-classify/checkpoint/checkpoint.pt


In [8]:
prepare_dataset = True

## Prepare dataset

In [9]:
raw_data_dir = "tmp"
processed_out_dir = os.path.join(raw_data_dir, "processd")

if os.path.exists(processed_out_dir):
    shutil.rmtree(processed_out_dir)

os.makedirs(processed_out_dir, exist_ok=True)


In [10]:
s3_uri_train

's3://sagemaker-us-east-2-324346001917/bert-sst2-classify/data/train.csv'

In [11]:
from utils.sst2_split_utils import SST2SplitUtils
from s3_util import S3Util

if prepare_dataset:
    input_file = os.path.join(raw_data_dir , "datasetSentences.txt")
    split_file = os.path.join(raw_data_dir , "datasetSplit.txt")
    dictionary = os.path.join(raw_data_dir , "dictionary.txt")
    label_file = os.path.join(raw_data_dir , "sentiment_labels.txt")
    SST2SplitUtils().split( input_file, split_file, dictionary, label_file, processed_out_dir)
    
    S3Util().upload_file(os.path.join(processed_out_dir, "train.csv"), s3_uri_train )
    S3Util().upload_file(os.path.join(processed_out_dir, "dev.csv"), s3_uri_val )
    S3Util().upload_file(os.path.join(processed_out_dir, "test.csv"), s3_uri_test )
    
    

2021-01-19 23:07:30,157 - utils.sst2_split_utils - WARNING - Text not found in dictionary: But in Imax 3-D , the clichÃ©s disappear into the vertiginous perspectives opened up by the photography .
2021-01-19 23:07:30,158 - utils.sst2_split_utils - WARNING - Text not found in dictionary: -LRB- But it 's -RRB- worth recommending because of two marvelous performances by Michael Caine and Brendan Fraser .
2021-01-19 23:07:30,161 - utils.sst2_split_utils - WARNING - Text not found in dictionary: JirÃ­ Hubac 's script is a gem .
2021-01-19 23:07:30,162 - utils.sst2_split_utils - WARNING - Text not found in dictionary: You would n't call The Good Girl a date movie -LRB- an anti-date movie is more like it -RRB- , but when it 's good , it 's good and horrid .
2021-01-19 23:07:30,164 - utils.sst2_split_utils - WARNING - Text not found in dictionary: An incendiary , deeply thought-provoking look at one of the most peculiar -LRB- and peculiarly venomous -RRB- bigotries in our increasingly frighten

2021-01-19 23:07:30,275 - utils.sst2_split_utils - WARNING - Text not found in dictionary: U.S. audiences may find -LRB- Attal and Gainsbourg 's -RRB- unfamiliar personas give the film an intimate and quaint reality that is a little closer to human nature than what Hollywood typically concocts .
2021-01-19 23:07:30,279 - utils.sst2_split_utils - WARNING - Text not found in dictionary: Dazzles with its fully-written characters , its determined stylishness -LRB- which always relates to characters and story -RRB- and Johnny Dankworth 's best soundtrack in years .
2021-01-19 23:07:30,283 - utils.sst2_split_utils - WARNING - Text not found in dictionary: The band 's courage in the face of official repression is inspiring , especially for aging hippies -LRB- this one included -RRB- .
2021-01-19 23:07:30,285 - utils.sst2_split_utils - WARNING - Text not found in dictionary: -LRB- Lawrence bounces -RRB- all over the stage , dancing , running , sweating , mopping his face and generally displayi

2021-01-19 23:07:30,338 - utils.sst2_split_utils - WARNING - Text not found in dictionary: Too damn weird to pass up , and for the blacklight crowd , way cheaper -LRB- and better -RRB- than Pink Floyd tickets .
2021-01-19 23:07:30,340 - utils.sst2_split_utils - WARNING - Text not found in dictionary: Writer\/director Alexander Payne -LRB- Election -RRB- and his co-writer Jim Taylor brilliantly employ their quirky and fearless ability to look American angst in the eye and end up laughing .
2021-01-19 23:07:30,341 - utils.sst2_split_utils - WARNING - Text not found in dictionary: The reason to see `` Sade '' lay with the chemistry and complex relationship between the marquis -LRB- Auteil -RRB- and Emilie -LRB- Le Besco -RRB- .
2021-01-19 23:07:30,343 - utils.sst2_split_utils - WARNING - Text not found in dictionary: -LRB- Reno -RRB- delivers a monologue that manages to incorporate both the horror and the absurdity of the situation in a well-balanced fashion .
2021-01-19 23:07:30,344 - ut

2021-01-19 23:07:30,384 - utils.sst2_split_utils - WARNING - Text not found in dictionary: -LRB- Woo 's -RRB- most resonant film since The Killer .
2021-01-19 23:07:30,387 - utils.sst2_split_utils - WARNING - Text not found in dictionary: -LRB- Fincher 's -RRB- camera sense and assured pacing make it an above-average thriller .
2021-01-19 23:07:30,388 - utils.sst2_split_utils - WARNING - Text not found in dictionary: Exploits -LRB- headbanger -RRB- stereotypes in good fun , while adding a bit of heart and unsettling subject matter .
2021-01-19 23:07:30,389 - utils.sst2_split_utils - WARNING - Text not found in dictionary: Combines improbable melodrama -LRB- gored bullfighters , comatose ballerinas -RRB- with subtly kinky bedside vigils and sensational denouements , and yet at the end , we are undeniably touched .
2021-01-19 23:07:30,390 - utils.sst2_split_utils - WARNING - Text not found in dictionary: -LRB- A -RRB- devastatingly powerful and astonishingly vivid Holocaust drama .
2021-

2021-01-19 23:07:30,428 - utils.sst2_split_utils - WARNING - Text not found in dictionary: -LRB- D -RRB- espite its familiar subject matter , Ice Age is consistently amusing and engrossing ...
2021-01-19 23:07:30,430 - utils.sst2_split_utils - WARNING - Text not found in dictionary: The ingenious construction -LRB- adapted by David Hare from Michael Cunningham 's novel -RRB- constantly flows forwards and back , weaving themes among three strands which allow us to view events as if through a prism
2021-01-19 23:07:30,433 - utils.sst2_split_utils - WARNING - Text not found in dictionary: -LRB- Ramsay -RRB- visually transforms the dreary expanse of dead-end distaste the characters inhabit into a poem of art , music and metaphor .
2021-01-19 23:07:30,436 - utils.sst2_split_utils - WARNING - Text not found in dictionary: A coming-of-age film that avoids the cartoonish clichÃ©s and sneering humor of the genre as it provides a fresh view of an old type -- the uncertain girl on the brink of wo

...

2021-01-19 23:07:30,870 - utils.sst2_split_utils - WARNING - Text not found in dictionary: -LRB- Nelson 's -RRB- movie about morally compromised figures leaves viewers feeling compromised , unable to find their way out of the fog and the ashes .
2021-01-19 23:07:30,872 - utils.sst2_split_utils - WARNING - Text not found in dictionary: A gob of drivel so sickly sweet , even the eager consumers of Moore 's pasteurized ditties will retch it up like rancid crÃ¨me brÃ»lÃ©e .
2021-01-19 23:07:30,877 - utils.sst2_split_utils - WARNING - Text not found in dictionary: A baffling misfire , and possibly the weakest movie -LRB- Woody Allen -RRB- has made in the last twenty years .
2021-01-19 23:07:30,878 - utils.sst2_split_utils - WARNING - Text not found in dictionary: If I Spy were funny -LRB- enough -RRB- or exciting -LRB- enough -RRB- then it would be fairly simple to forgive the financial extortion it 's trying to reap from the moviegoing public .
2021-01-19 23:07:30,882 - utils.sst2_split_ut

2021-01-19 23:07:30,921 - utils.sst2_split_utils - WARNING - Text not found in dictionary: Her film is like a beautiful food entrÃ©e that is n't heated properly , so that it ends up a bit cold and relatively flavorless .
2021-01-19 23:07:30,922 - utils.sst2_split_utils - WARNING - Text not found in dictionary: The thought of watching this film with an audience full of teenagers fixating on its body humour and reinforcement of stereotypes -LRB- of which they 'll get plenty -RRB- fills me with revulsion .
2021-01-19 23:07:30,924 - utils.sst2_split_utils - WARNING - Text not found in dictionary: -LRB- Taylor -RRB- takes us on a ride that 's consistently surprising , easy to watch -- but , oh , so dumb .
2021-01-19 23:07:30,925 - utils.sst2_split_utils - WARNING - Text not found in dictionary: Though Jones and Snipes are enthralling , the movie bogs down in rhetoric and clichÃ© .
2021-01-19 23:07:30,925 - utils.sst2_split_utils - WARNING - Text not found in dictionary: The most remarkable 

2021-01-19 23:07:30,994 - utils.sst2_split_utils - WARNING - Text not found in dictionary: Lush and beautifully photographed -LRB- somebody suggested the stills might make a nice coffee table book -RRB- , but ultimately you 'll leave the theater wondering why these people mattered .
2021-01-19 23:07:31,000 - utils.sst2_split_utils - WARNING - Text not found in dictionary: `` I blame all men for war , '' -LRB- the warden 's daughter -RRB- tells her father .
2021-01-19 23:07:31,002 - utils.sst2_split_utils - WARNING - Text not found in dictionary: This is rote spookiness , with nary an original idea -LRB- or role , or edit , or score , or anything , really -RRB- in sight , and the whole of the proceedings beg the question ` Why ? '
2021-01-19 23:07:31,002 - utils.sst2_split_utils - WARNING - Text not found in dictionary: The type of dumbed-down exercise in stereotypes that gives the -LRB- teen comedy -RRB- genre a bad name .
2021-01-19 23:07:31,003 - utils.sst2_split_utils - WARNING - Te

2021-01-19 23:07:31,081 - utils.sst2_split_utils - WARNING - Text not found in dictionary: The pace of the film is very slow -LRB- for obvious reasons -RRB- and that too becomes off-putting .
2021-01-19 23:07:31,082 - utils.sst2_split_utils - WARNING - Text not found in dictionary: -LRB- T -RRB- he ideas of Revolution # 9 are more compelling than the execution
2021-01-19 23:07:31,083 - utils.sst2_split_utils - WARNING - Text not found in dictionary: -LRB- A -RRB- stale retread of the '53 original .
2021-01-19 23:07:31,084 - utils.sst2_split_utils - WARNING - Text not found in dictionary: For all of its insights into the dream world of teen life , and its electronic expression through cyber culture , the film gives no quarter to anyone seeking to pull a cohesive story out of its 2Â 1\/2 - hour running time .
2021-01-19 23:07:31,085 - utils.sst2_split_utils - WARNING - Text not found in dictionary: Stale and clichÃ©d to a fault .
2021-01-19 23:07:31,085 - utils.sst2_split_utils - WARNING

2021-01-19 23:07:31,118 - utils.sst2_split_utils - WARNING - Text not found in dictionary: A rip-off twice removed , modeled after -LRB- Seagal 's -RRB- earlier copycat Under Siege , sometimes referred to as Die Hard on a boat .
2021-01-19 23:07:31,119 - utils.sst2_split_utils - WARNING - Text not found in dictionary: Does n't deserve a passing grade -LRB- even on a curve -RRB- .
2021-01-19 23:07:31,119 - utils.sst2_split_utils - WARNING - Text not found in dictionary: -LRB- Lee -RRB- treats his audience the same way that Jim Brown treats his women -- as dumb , credulous , unassuming , subordinate subjects .
2021-01-19 23:07:31,120 - utils.sst2_split_utils - WARNING - Text not found in dictionary: Ultimately this is a frustrating patchwork : an uneasy marriage of Louis Begley 's source novel -LRB- About Schmidt -RRB- and an old Payne screenplay .
2021-01-19 23:07:31,121 - utils.sst2_split_utils - WARNING - Text not found in dictionary: Marinated in clichÃ©s and mawkish dialogue .
2021-

## Train

This shows you how to train BERT on SageMaker using SPOT instances

In [12]:
inputs_full =  {
    "train" : s3_uri_train,
    "val" : s3_uri_val,

}



inputs = inputs_full

In [13]:
sm_localcheckpoint_dir="/opt/ml/checkpoints/"

In [14]:
instance_type = "ml.p3.2xlarge"
instance_type_gpu_map = {"ml.p3.8xlarge":4, "ml.p3.2xlarge": 1, "ml.p3.16xlarge":8}

In [15]:
hp = {
"epochs" : 30,
"earlystoppingpatience" : 3,
# Increasing batch size might end up with CUDA OOM error, increase grad accumulation instead
"batch" : 8 * instance_type_gpu_map[instance_type],
"trainfile" :s3_uri_train.split("/")[-1],
"valfile" : s3_uri_val.split("/")[-1],
"datasetfactory":"datasets.sst2_dataset_factory.SST2DatasetFactory",
# The number of steps to accumulate gradients for
"gradaccumulation" : 4,
"log-level":"INFO",
# This param depends on your model max pos embedding size or when large you might end up with CUDA OOM error    
"maxseqlen" : 512,
# Make sure the lr is quite small, as this is a pretrained model..
"lr":0.00001,
# Use finetuning (set to 1), if you only want to change the weights in the final classification layer.. 
"finetune": 0,
"checkpointdir" : sm_localcheckpoint_dir,
# Checkpoints once every n epochs
"checkpointfreq": 2
}



In [16]:
hp

{'epochs': 30,
 'earlystoppingpatience': 3,
 'batch': 8,
 'trainfile': 'train.csv',
 'valfile': 'dev.csv',
 'datasetfactory': 'datasets.sst2_dataset_factory.SST2DatasetFactory',
 'gradaccumulation': 4,
 'log-level': 'INFO',
 'maxseqlen': 512,
 'lr': 1e-05,
 'finetune': 0,
 'checkpointdir': '/opt/ml/checkpoints/',
 'checkpointfreq': 2}

In [17]:
inputs

{'train': 's3://sagemaker-us-east-2-324346001917/bert-sst2-classify/data/train.csv',
 'val': 's3://sagemaker-us-east-2-324346001917/bert-sst2-classify/data/dev.csv'}

In [18]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainScore",
                     "Regex": "###score: train_score### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationScore",
                     "Regex": "###score: val_score### (\d*[.]?\d*)"}
                    ]

In [19]:
# set True if you need spot instance
use_spot = True
train_max_run_secs =   2*24 * 60 * 60
spot_wait_sec =  5 * 60
max_wait_time_secs = train_max_run_secs +  spot_wait_sec

if not use_spot:
    max_wait_time_secs = None
    s3_checkpoint=None
    sm_localcheckpoint_dir=None
    hp.pop("checkpointdir")
    
# During local mode, no spot.., use smaller dataset
if instance_type == 'local':
    use_spot = False
    max_wait_time_secs = 0
    wait = True
    # Use smaller dataset to run locally
    inputs = inputs_sample
    
    


In [20]:
job_type = "bert-sst2-classification"
base_name = "{}".format(job_type)

In [21]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
                    entry_point='main.py',
                    source_dir = 'src',
                    role=role,
                    framework_version ="1.4.0",
                    py_version='py3',
                    instance_count=1,
                    instance_type=instance_type,
                    hyperparameters = hp,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    volume_size=30,
                    code_location=s3_code_path,
                    debugger_hook_config=False,
                    base_job_name =base_name,  
                    use_spot_instances = use_spot,
                    max_run =  train_max_run_secs,
                    max_wait = max_wait_time_secs,   
                    checkpoint_s3_uri=s3_checkpoint,
                    checkpoint_local_path=sm_localcheckpoint_dir)

estimator.fit(inputs, wait=True)

2021-01-19 23:07:57,962 - sagemaker - INFO - Creating training-job with name: bert-sst2-classification-2021-01-19-12-07-55-301
2021-01-19 12:07:59 Starting - Starting the training job...
2021-01-19 12:08:01 Starting - Launching requested ML instances...
2021-01-19 12:09:04 Starting - Preparing the instances for training......
2021-01-19 12:10:16 Downloading - Downloading input data...
2021-01-19 12:10:52 Training - Downloading the training image......
2021-01-19 12:12:04 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-01-19 12:12:05,412 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-01-19 12:12:05,439 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-01-19 12:12:06,897 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-01-19 12:12:07,3

2021-01-19 12:12:28,692 - transformers.file_utils - INFO - copying /tmp/tmp7yc_rzmn to cache at /root/.cache/torch/transformers/35d8b9d36faaf46728a0192d82bf7d00137490cd6074e8500778afed552a67e5.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2
2021-01-19 12:12:29,242 - transformers.file_utils - INFO - creating metadata file for /root/.cache/torch/transformers/35d8b9d36faaf46728a0192d82bf7d00137490cd6074e8500778afed552a67e5.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2
2021-01-19 12:12:29,242 - transformers.file_utils - INFO - removing temp file /tmp/tmp7yc_rzmn
2021-01-19 12:12:29,319 - transformers.modeling_utils - INFO - loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-pytorch_model.bin from cache at /root/.cache/torch/transformers/35d8b9d36faaf46728a0192d82bf7d00137490cd6074e8500778afed552a67e5.3fadbea36527ae472139fe84cddaa65454d7429f12d543d80bfc3ad70de55ac2
2021-01-19 12:12:32,843 - transformers.modeling_utils


2021-01-19 12:47:29 Uploading - Uploading generated training model
2021-01-19 12:48:26 Completed - Training job completed
Training seconds: 2290
Billable seconds: 687
Managed Spot Training savings: 70.0%


## Deploy BERT model

#### Inference container
Ideally the server containing should already have all the required dependencies installed to reduce start up time and ensure that the runtime enviornment is consistent. This can be implemented using a custom docker image.

But for this demo, to simplify, we will let the Pytorch container script model install the dependencies during start up. As a result, you will see some of the initial ping requests fail, until all dependencies are installed.


In [22]:
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role

model_uri = estimator.model_data

model = PyTorchModel(model_data=model_uri,
                     role=role,
                     py_version = "py3",
                     framework_version='1.4.0',
                     entry_point='serve.py',
                     source_dir='src')

predictor = model.deploy(initial_instance_count=1, instance_type='ml.p3.2xlarge')

2021-01-20 00:02:00,904 - sagemaker - INFO - Creating model with name: pytorch-inference-2021-01-19-13-02-00-904
2021-01-20 00:02:04,616 - sagemaker - INFO - Creating endpoint with name pytorch-inference-2021-01-19-13-02-02-496
----------------!

### Invoke API

In [23]:
import json


class TextSerDes:
    
     def serialize(self, x):
        data_bytes="\n".join(x).encode("utf-8")
        return data_bytes
    
     def deserialize(self, x, content_type):
        payload =   x.read().decode("utf-8")
        return json.loads(payload) 

In [24]:
def chunk_predict(predictor, data, chunk_size=50):
    predictor.serializer = TextSerDes()
    predictor.deserializer = TextSerDes()
    
    result = []
    for i in range(0, len(data), chunk_size):
        
        re = predictor.predict(data[i:i+chunk_size],  initial_args={ "Accept":"text/json", "ContentType" : "text/csv" })
        result.extend(re)
    return result

In [25]:
from s3_util import S3Util
import csv, io

def load_test_csv(s3_uri):
    os.path.join(processed_out_dir, "test.csv")
    data = S3Util().download_object(s3_uri).decode("utf-8")
    
    csv_reader = csv.reader(io.StringIO(data), delimiter='\t',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    actuals =[]
    inputs = []
    ids = []
    for r in csv_reader:
        text = r[0]
        label = r[1]
        id = r[2]
        
        inputs.append(text)
        actuals.append(label)
        ids.append(id)
    return inputs, actuals, ids
        

def write_predictions_csv(test_data, predictions, output_file):
    
    with open(output_file, "w") as f:
        csv_writer = csv.writer(f, delimiter='|',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
            
        for d,p in zip(test_data, predictions):
            csv_writer.writerow([d,p])


In [26]:

test_data, test_labels, test_ids = load_test_csv(s3_uri_test)
response  = chunk_predict(predictor, test_data )
predictions_label = [ list(l.keys())[0] for l in response ]

write_predictions_csv(test_ids, predictions_label, "sst2-output.csv" )

## Delete endpoint

In [27]:
predictor.delete_endpoint()

2021-01-20 00:11:21,424 - sagemaker - INFO - Deleting endpoint configuration with name: pytorch-inference-2021-01-19-13-02-02-496
2021-01-20 00:11:22,783 - sagemaker - INFO - Deleting endpoint with name: pytorch-inference-2021-01-19-13-02-02-496
